In [1]:
import coiled
cluster = coiled.Cluster(n_workers=25)

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found
Dashboard: http://ec2-3-133-112-54.us-east-2.compute.amazonaws.com:8787


/opt/conda/lib/python3.8/site-packages/distributed/client.py:1129: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+-----------+
| Package     | client | scheduler | workers   |
+-------------+--------+-----------+-----------+
| blosc       | None   | 1.10.2    | 1.10.2    |
| dask        | 2.30.0 | 2021.01.1 | 2021.01.1 |
| distributed | 2.30.1 | 2021.01.1 | 2021.01.1 |
| lz4         | None   | 3.1.3     | 3.1.3     |
| msgpack     | 1.0.0  | 1.0.2     | 1.0.2     |
| numpy       | 1.19.4 | 1.19.5    | 1.19.5    |
+-------------+--------+-----------+-----------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [2]:
client

Client Scheduler: tls://ec2-3-133-112-54.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-3-133-112-54.us-east-2.compute.amazonaws.com:8787,Cluster Workers: 25 Cores: 100 Memory: 429.50 GB


In [4]:
import dask_geopandas
import geopandas as gpd
import pandas as pd
import dask.dataframe as dd
import numpy as np
from dask.distributed import Client, LocalCluster
import pygeos

In [5]:
def dask_dissolve(ddf, by=None, aggfunc="first"):
    """
    ddf should be sorted according to `by` for the best performance
    
    n_partitoins should be approximately workers * 4
    
    In case geometry columns is no called "geometry" it needs to be reassigned after `.compute()`
    """    
    def _partition_dissolve(partition, by=None, aggfunc="first", geometry_name="geometry"):
        
        if by is None:
            by = np.zeros(len(partition), dtype="int64")

        # Process non-spatial component
        data = partition.drop(labels=geometry_name, axis=1)
        aggregated_data = data.groupby(by=by).agg(aggfunc)

        # Process spatial component
        def merge_geometries(block):
            merged_geom = block.unary_union
            return merged_geom

        g = partition.groupby(by=by, group_keys=False)[geometry_name].agg(
            merge_geometries
        )

        # Aggregate
        aggregated_geometry = gpd.GeoDataFrame(g, geometry=geometry_name, crs=partition.crs)
        # Recombine
        aggregated = aggregated_geometry.join(aggregated_data)

        aggregated = aggregated.reset_index()

        return aggregated
    
    geometry_name = ddf.geometry.name
    cols = ddf.columns.to_list()
    cols.remove(geometry_name)
    
    if by is None:
        cols.insert(0, 'index')
        cols.insert(1, geometry_name)
    else:
        cols.remove(by)
        cols.insert(0, by)
        cols.insert(1, geometry_name)
    
    first_step = ddf.map_partitions(_partition_dissolve, by=by, geometry_name=geometry_name, aggfunc=aggfunc, meta=gpd.GeoDataFrame(columns=cols))
    
    if by is None:
        pre_second = first_step.set_index('index')
    else:
        pre_second = first_step.set_index(by)
    
    second_step = pre_second.map_partitions(_partition_dissolve, by=by, geometry_name=geometry_name, aggfunc=aggfunc, meta=gpd.GeoDataFrame(columns=cols))
    
    return second_step

In [6]:
tess = gpd.read_parquet("../../../urbangrammar_samba/sp_sig_concept/barcelona/tess.pq", columns=['tessellation']).rename_geometry("geometry")

In [7]:
clusters = pd.read_csv("../../../urbangrammar_samba/sp_sig_concept/barcelona/FINAL_cluster_labels.csv", index_col=0)

In [8]:
tess['cluster'] = clusters.clusters.values

In [9]:
%%time
ddf = dask_geopandas.from_geopandas(tess, npartitions=64)
final = dask_dissolve(ddf, by='cluster').compute()

AttributeError: module 'dask_geopandas' has no attribute 'from_geopandas'

In [10]:
client.close()

In [12]:
cluster.close()